In [ ]:
import os
import argparse
import Augmentor
import shutil
import time


# https://github.com/mdbloice/Augmentor
# pip install Augmentor
# 機械学習用にイメージデータを水増しするライブラリ

# 再帰フォルダ探索のPATH格納
IMG_LIST_PATH = []
# 増やしたい画像枚数
TEST_DATA_NUM = 1000


def augment_data(img_dic):
    # Augmentorの処理
    for path, num in img_dic.items():
        make_test_img_num = TEST_DATA_NUM - num

        # 画像フォルダ
        p = Augmentor.Pipeline(path)

        # キャンバスの歪み
        p.skew_tilt(probability=0.3, magnitude=0.5)

        # 中心の歪み
        p.random_distortion(probability=0.3, grid_width=2, grid_height=2, magnitude=2)

        # 回転
        p.rotate90(probability=0.3)
        p.rotate270(probability=0.3)
        p.rotate(probability=0.3, max_left_rotation=10, max_right_rotation=10)

        # 反転
        p.flip_left_right(probability=0.3)
        p.flip_top_bottom(probability=0.3)

        # ずらし
        p.crop_random(probability=1, percentage_area=0.3)

        p.resize(probability=1.0, width=64, height=64)
        p.sample(make_test_img_num)


def traverse_dir(path):
    for file_or_dir in os.listdir(path):
        abs_path = os.path.abspath(os.path.join(path, file_or_dir))
        if os.path.isdir(abs_path):
            traverse_dir(abs_path)
        else:
            # 画像を見つけたら、親フォルダのPathを格納しておく
            img_directory = os.path.dirname(abs_path)
            if img_directory not in IMG_LIST_PATH:
                IMG_LIST_PATH.append(img_directory)


def get_folder_list(path):
    global IMG_LIST_PATH

    # 再帰でフォルダ内のDatasetを取得
    traverse_dir(path)
    imglist_num = {}
    # DatasetのFolderPathと、Folder内の画像数をDictionaryに入れておく
    for path in IMG_LIST_PATH:
        imglist_num[path] = len(os.listdir(path))

    IMG_LIST_PATH = {}
    return imglist_num


def copy_original_data(imgs_dic):
    for path in imgs_dic.keys():
        # Augmentorの処理まち
        while True:
            check_foleder = os.path.join(path, "output")
            if os.path.exists(check_foleder):
                time.sleep(1)
                break

        # label直下に持ってくる
        test_data_path = os.path.join(path, "output")
        for file_or_dir in os.listdir(test_data_path):
            abs_path = os.path.abspath(os.path.join(test_data_path, file_or_dir))
            if os.path.isdir(abs_path):
                pass
            else:
                dest_path= os.path.join(path, file_or_dir)
                shutil.move(abs_path, dest_path)

        del_folder = os.path.join(path, "output")
        shutil.rmtree(del_folder)


def main():
    # 一括でAugmentorを行いたいFolderを指定する
    target_path = "/Users/shihosato/OneDrive/programming/tsuda_lab/cancer_cells/cancer_images"

    # 上記指定フォルダ配下のフォルダに存在するもの全部を取得
    imgs_dic = get_folder_list(target_path)
    # Augumentorを一括で実施
    augment_data(imgs_dic)
    # Augmentorがoutput folderを作るので、同一フォルダにコピーしてくる
    copy_original_data(imgs_dic)

if __name__ == "__main__":
    main()

In [ ]:
#各画像typeごとにディレクトリを作成し、画像データを格納した後👇

#撮影したデータについてラベリング（画像データと商品名の紐付け）を実施し、学習/検証データを用意する、というコード


#ラベリングによる学習/検証データの準備

from PIL import Image
import os, glob
import numpy as np
import random, math

#画像が保存されているルートディレクトリのパス
root_dir  = "/Users/shihosato/OneDrive/programming/tsuda_lab/cancer_cells/cancer_images/" #MacとWindowsでは違うので書き換えを
# がん細胞の種類
categories = ["type0", "type1", "type2", "type3", "type4", "type5", "typeB"]

# 画像データ用配列
X = []
# ラベルデータ用配列
Y = []

#画像データごとにadd_sample()を呼び出し、X,Yの配列を返す関数
def make_sample(files):
    global X, Y
    X = []
    Y = []
    for cat, fname in files:
        add_sample(cat, fname)
    return np.array(X), np.array(Y) #Numpy配列

#渡された画像データを読み込んでXに格納し、また、
#画像データに対応するcategoriesのidxをY格納する関数
def add_sample(cat, fname):
    img = Image.open(fname)
    img = img.convert("RGB")
    img = img.resize((70, 70))
    data = np.asarray(img)
    X.append(data)
    Y.append(cat)

#全データ格納用配列
allfiles = []

#カテゴリ配列の各値と、それに対応するidxを認識し、全データをallfilesにまとめる
for idx, cat in enumerate(categories):
    image_dir = root_dir + "/" + cat
    files = glob.glob(image_dir + "/*.tiff") #拡張子check
    for f in files:
        allfiles.append((idx, f))

#シャッフル後、学習データと検証データに分ける
random.shuffle(allfiles)
th = math.floor(len(allfiles) * 0.8)
train = allfiles[0:th]
test  = allfiles[th:]
X_train, y_train = make_sample(train)
X_test, y_test = make_sample(test)
xy = (X_train, X_test, y_train, y_test)

#データを保存する
np.save("cancercells_npdata", xy) #cancercells_npdata.npy

In [ ]:
#モデルの構築

from keras import layers, models

model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation="relu",input_shape=(70,70,3))) #一番上のcellにある"img = img.resize((150, 150))"と揃える
model.add(layers.MaxPooling2D((2,2))) #MaxPooling2D layer...プーリング層
model.add(layers.Conv2D(64,(3,3),activation="relu")) #Conv2D layer...畳み込み層
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten()) #後に多層パーセプトロンに入力するときはデータをフラット化する必要がある。
#4次元配列を1次元配列に変換するにはFlatten()という層を追加するだけでOK。ユニット数などは自動的に計算してくれる。
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512,activation="relu")) #Dense layer...全結合層
model.add(layers.Dense(7,activation="sigmoid")) #分類先の種類分設定(今回7分類なので7とする） #出力数

#モデル構成の確認
model.summary()

In [ ]:
#モデルのコンパイル

from keras import optimizers

model.compile(loss="binary_crossentropy",
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=["acc"])

In [ ]:
#先ほど作成した学習/検証データを読み込み、データの正規化といった学習に向けた準備をする

#データの準備
from keras.utils import np_utils
import numpy as np

categories = ["type0", "type1", "type2", "type3", "type4", "type5", "typeB"]
nb_classes = len(categories)

X_train, X_test, y_train, y_test = np.load("cancercells_npdata.npy")

#データの正規化
X_train = X_train.astype("float") / 255
X_test  = X_test.astype("float")  / 255

#kerasで扱えるようにcategoriesをベクトルに変換
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test  = np_utils.to_categorical(y_test, nb_classes)

In [ ]:
#モデルの学習
model = model.fit(X_train,
                  y_train,
                  epochs=10,
                  batch_size=128,
                  validation_data=(X_test,y_test))

In [ ]:
#学習結果を表示

import matplotlib.pyplot as plt

acc = model.history['acc']
val_acc = model.history['val_acc']
loss = model.history['loss']
val_loss = model.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.savefig('cancerimages_accuracy6') #毎回番号変えて保存

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.savefig('cancerimages_loss6') #毎回番号変えて保存

In [ ]:
#モデルの保存

json_string = model.model.to_json()
open('/Users/shihosato/OneDrive/programming/tsuda_lab/cancer_cells/canerimages_predict.json', 'w').write(json_string)

#重みの保存

hdf5_file = "/Users/shihosato/OneDrive/programming/tsuda_lab/cancer_cells/cancerimages_predict.hdf5"
model.model.save_weights(hdf5_file)

In [ ]:
#テストデータの作成

from PIL import Image
import os, glob
import numpy as np
import random, math

# 画像が保存されているディレクトリのパス
root_dir = "/Users/shihosato/OneDrive/programming/tsuda_lab/cancer_cells/cancer_images"
# 画像が保存されているフォルダ名
categories = ["type0", "type1", "type2", "type3", "type4", "type5", "typeB"]

X = [] # 画像データ
Y = [] # ラベルデータ

# フォルダごとに分けられたファイルを収集
#（categoriesのidxと、画像のファイルパスが紐づいたリストを生成）
allfiles = []
for idx, cat in enumerate(categories):
    image_dir = root_dir + "/" + cat
    files = glob.glob(image_dir + "/*.tiff")
    for f in files:
        allfiles.append((idx, f))

for cat, fname in allfiles:
    img = Image.open(fname)
    img = img.convert("RGB")
    img = img.resize((70, 70))
    data = np.asarray(img)
    X.append(data)
    Y.append(cat)

x = np.array(X)
y = np.array(Y)

np.save("/Users/shihosato/OneDrive/programming/tsuda_lab/cancer_cells/cancer_data_test/cancer_data_test_X_150", x)
np.save("/Users/shihosato/OneDrive/programming/tsuda_lab/cancer_cells/cancer_data_test/cancer_data_test_Y_150", y)

In [ ]:
# モデルの精度を測る

#評価用のデータの読み込み
eval_X = np.load("/Users/shihosato/OneDrive/programming/tsuda_lab/cancer_cells/cancer_data_test/cancer_data_test_X_150.npy")
eval_Y = np.load("/Users/shihosato/OneDrive/programming/tsuda_lab/cancer_cells/cancer_data_test/cancer_data_test_Y_150.npy")

#Yのデータをone-hotに変換
from keras.utils import np_utils

eval_Y = np_utils.to_categorical(eval_Y, 7)

score = model.model.evaluate(x=eval_X, y=eval_Y)

print('loss=', score[0])
print('accuracy=', score[1])

### 結果

#### そのまま
* accuracy=0.87ぐらい（記録忘れ）

#### Dropout0.5, Augmentorで1000画像/1カテゴリに
* 6987/6987  - 38s 5ms/step  
* loss= 3.289973085592348  
* accuracy= 0.788239433356957
* plt.savefig('cancerimages_accuracy')（以降.1, .2,,,として保存した）
* plt.savefig('cancerimages_loss')（同上）

#### img = img.resize((150, 150))→((250, 250))（1）
* validationすら0.85程度であったため中止

#### batch_size=6→32（2）
* Epoch 1/10 5589/5589 - 257s 46ms/step - loss: 0.4092 - acc: 0.8554 - val_loss: 0.3811 - val_acc: 0.8568
* Epoch 10/10 5589/5589- 233s 42ms/step - loss: 0.3190 - acc: 0.8680 - val_loss: 0.3892 - val_acc: 0.8414（学習回数重ねたのに下がっている）
* 6987/6987 - 101s 14ms/step
* loss= 3.879581622053186
* accuracy= 0.7531743155062173

####  img = img.resize((100, 100)) (上書き保存したため無くなった）
* Epoch 1/10 5589/5589 - 48s 9ms/step - loss: 0.4210 - acc: 0.8550 - val_loss: 0.3968 - val_acc: 0.8571
* Epoch 10/10 5589/5589 - 47s 8ms/step - loss: 0.3523 - acc: 0.8589 - val_loss: 0.3451 - val_acc: 0.8623
* 6987/6987 - 20s 3ms/step
* loss= 3.1761112101893234
* accuracy= 0.7909383186543979

#### batch_size=64（3）
* Epoch 1/10 5589/5589 - 41s 7ms/step - loss: 0.4333 - acc: 0.8525 - val_loss: 0.4168 - val_acc: 0.8571
* Epoch 10/10 5589/5589 - 40s 7ms/step - loss: 0.3639 - acc: 0.8583 - val_loss: 0.3662 - val_acc: 0.8585
* 6987/6987 - 20s 3ms/step
* loss= 3.3677486562448937
* accuracy= 0.7690200838678228

#### batch_size=128（4）
* Epoch 1/10 5589/5589 - 37s 7ms/step - loss: 0.4471 - acc: 0.8469 - val_loss: 0.4210 - val_acc: 0.8571
* Epoch 10/10 5589/5589 - 35s 6ms/step - loss: 0.3713 - acc: 0.8571 - val_loss: 0.3759 - val_acc: 0.8579
* 6987/6987 - 20s 3ms/step
* loss= 2.302585364104648
* accuracy= 0.8571428656578064
* plotが今までで一番綺麗

#### img = img.resize((50, 50)) （5）
* Epoch 1/10 5589/5589 - 12s 2ms/step - loss: 0.4850 - acc: 0.8341 - val_loss: 0.4227 - val_acc: 0.8571
* Epoch 10/10 5589/5589 - 11s 2ms/step - loss: 0.3878 - acc: 0.8575 - val_loss: 0.3785 - val_acc: 0.8588
* 6987/6987 - 5s 719us/step
* loss= 2.3025854514599597
* accuracy= 0.8571428656578064
* 4と結果が同じ

#### img = img.resize((70, 70)) （6）
* Epoch 1/10 5589/5589 - 20s 4ms/step - loss: 0.4686 - acc: 0.8399 - val_loss: 0.4193 - val_acc: 0.8571
* Epoch 10/10 5589/5589 - 19s 3ms/step - loss: 0.3809 - acc: 0.8572 - val_loss: 0.3755 - val_acc: 0.8558
* 6987/6987 - 11s 2ms/step
* loss= 2.3025853630127067
* accuracy= 0.8571428656578064
* imgsize((100, 100))~((50, 50))間で結果は変らないのだろうか

#### img = img.resize((50, 50)) , epoc=15
* Epoch 1/15 5589/5589 - 11s 2ms/step - loss: 0.4723 - acc: 0.8492 - val_loss: 0.4207 - val_acc: 0.8571
* Epoch 15/15 5589/5589 - 11s 2ms/step - loss: 0.3813 - acc: 0.8573 - val_loss: 0.3818 - val_acc: 0.8569
* 6987/6987 - 5s 705us/step
* loss= 2.3025853630127067
* accuracy= 0.8571428656578064
* 結果に変化無し